In [1]:
%pip install networkx plotly ipywidgets pandas anywidget nbformat>=4.2.0


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /home/pienskoi/PycharmProjects/TRSKS/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import networkx as nx
import plotly.graph_objects as go
from ipywidgets import widgets, VBox, HBox, Layout

In [30]:
CONFIG = {
    "su_count": 4,
    "spines_per_su": 4,
    "leaves_per_su": 8,
    "nodes_per_su": 32,
}

CONFIG['spine_count'] = CONFIG['spines_per_su'] * CONFIG['su_count']

In [33]:
def create_graph():
    G = nx.Graph()
    pos = {}
    node_props = {}

    SPINE_SPACING = 10
    SU_OFFSET = 40
    LEAF_SPACING = 5
    NODE_SPACING = 1.25

    for s_idx in range(1, CONFIG['spine_count'] + 1):
        spine_name = f"Spine-{s_idx:02d}"
        G.add_node(spine_name, type='spine', layer=3)
        pos[spine_name] = (s_idx * SPINE_SPACING, 3)
        node_props[spine_name] = {'color': '#000000', 'size': 24, 'label': spine_name}

    node_global_idx = 1
    for su_idx in range(1, CONFIG['su_count'] + 1):
        su_x_offset = (su_idx - 1) * SU_OFFSET

        for l_idx in range(1, CONFIG['leaves_per_su'] + 1):
            leaf_name = f"Leaf-{su_idx:02d}-{l_idx:02d}"
            G.add_node(leaf_name, type='leaf', layer=2)
            pos[leaf_name] = (su_x_offset + (l_idx * LEAF_SPACING), 2)
            node_props[leaf_name] = {'color': '#333333', 'size': 18, 'label': leaf_name}

            for i in range(1, CONFIG['spine_count'] + 1):
                spine_target = f"Spine-{i:02d}"
                G.add_edge(leaf_name, spine_target, color='#d62728', width=1)

        for n_idx in range(1, CONFIG['nodes_per_su'] + 1):
            node_name = f"H100-{node_global_idx:03d}"
            G.add_node(node_name, type='compute', layer=1)
            pos[node_name] = (su_x_offset + (n_idx * NODE_SPACING), 1)
            node_props[node_name] = {'color': '#8c735a', 'size': 12, 'label': node_name}

            for r_idx in range(1, CONFIG['leaves_per_su'] + 1):
                leaf_target = f"Leaf-{su_idx:02d}-{r_idx:02d}"
                G.add_edge(node_name, leaf_target, color='#ff7f0e', width=0.5)

            node_global_idx += 1

    return G, pos, node_props

class NetworkSimulation:
    def __init__(self):
        self.G, self.pos, self.node_props = create_graph()
        self.source_node = None
        self.target_nodes = []
        self.active_edges = []
        self.node_list = []

        self.setup_widgets()
        self.setup_figure()
        self.refresh_plot_data()

    def setup_widgets(self):
        self.metrics_display = widgets.HTML(
            value="Calculating metrics...",
            layout=Layout(width='300px', border='1px solid #ddd')
        )

        self.mode_dropdown = widgets.Dropdown(
            options=['Unicast', 'Broadcast', 'Multicast', 'Anycast'],
            value='Unicast',
            description='Routing:',
            layout=Layout(width='200px')
        )

        self.remove_toggle = widgets.ToggleButton(
            value=False,
            description='Maintenance Mode (Remove Node)',
            button_style='danger',
            icon='trash',
            layout=Layout(width='250px')
        )

        self.status_label = widgets.Label(value="Select a Source Node (Click on graph)")
        self.reset_btn = widgets.Button(description="Reset Sim", icon="refresh", layout=Layout(width='120px'))
        self.step_btn = widgets.Button(description="Next Step", icon="play", disabled=True, layout=Layout(width='120px'))

        self.reset_btn.on_click(self.reset_selection)
        self.step_btn.on_click(self.animate_step)
        self.mode_dropdown.observe(self.on_mode_change, names='value')
        self.remove_toggle.observe(self.on_remove_toggle, names='value')

    def update_metrics(self):
        G = self.G
        if G.number_of_nodes() == 0:
            self.metrics_display.value = "Граф порожній"
            return

        N = G.number_of_nodes()
        degrees = [d for n, d in G.degree()]
        S = max(degrees) if degrees else 0

        if nx.is_connected(G):
            D = nx.diameter(G)
            D2 = nx.average_shortest_path_length(G)
        else:
            D = float('inf')
            D2 = float('inf')

        Q = (N * S) / D2 if D2 != 0 and D2 != float('inf') else 0
        C = S * D if D != float('inf') else float('inf')

        html_template = f"""
        <h4 style="margin-top:0;">Topology metrics</h4>
        <table style="width:100%; border-collapse: collapse;">
            <tr><td><b>N</b> (nodes):</td><td style="text-align:right;">{N}</td></tr>
            <tr><td><b>S</b> (degree):</td><td style="text-align:right;">{S}</td></tr>
            <tr><td><b>D</b> (diameter):</td><td style="text-align:right;">{D}</td></tr>
            <tr><td><b style="text-decoration:overline">D</b> (avg diameter):</td><td style="text-align:right;">{D2:.2f}</td></tr>
            <tr><td><b>Q</b> (traffic):</td><td style="text-align:right;">{Q:.2f}</td></tr>
            <tr><td><b>C</b> (cost):</td><td style="text-align:right;">{C if C != float('inf') else '∞'}</td></tr>
        </table>
        """
        self.metrics_display.value = html_template

    def setup_figure(self):
        self.fig = go.FigureWidget()

        self.fig.add_trace(go.Scatter(
            x=[], y=[], mode='lines',
            line=dict(color='#bababa', width=1),
            hoverinfo='none', showlegend=False
        ))

        self.fig.add_trace(go.Scatter(
            x=[], y=[], mode='lines',
            line=dict(color='blue', width=3),
            name='Active Route', showlegend=False
        ))

        self.fig.add_trace(go.Scatter(
            x=[], y=[], mode='markers',
            marker=dict(size=[], color=[], line=dict(width=1, color='black')),
            text=[], hoverinfo='text', showlegend=False
        ))

        self.fig.add_trace(go.Scatter(
            x=[], y=[], mode='markers',
            marker=dict(size=30, color='rgba(0,0,0,0)', line=dict(color='red', width=3)),
            showlegend=False, hoverinfo='skip'
        ))

        self.fig.update_layout(
            title="Eos NVIDIA DGX SuperPOD Topology",
            template='plotly_white',
            xaxis=dict(visible=False), yaxis=dict(visible=False),
            width=1500,
            height=800,
            margin=dict(b=20,l=20,r=20,t=50),
            hovermode='closest', dragmode='pan',
            clickmode='event'
        )

        self.fig.data[2].on_click(self.handle_node_click)

    def refresh_plot_data(self):
        edge_x, edge_y = [], []
        for u, v in self.G.edges():
            if u in self.pos and v in self.pos:
                x0, y0 = self.pos[u]
                x1, y1 = self.pos[v]
                edge_x.extend([x0, x1, None])
                edge_y.extend([y0, y1, None])

        self.node_list = list(self.G.nodes())
        node_x, node_y, node_text, node_color, node_size = [], [], [], [], []

        for node in self.node_list:
            props = self.node_props[node]
            node_x.append(self.pos[node][0])
            node_y.append(self.pos[node][1])
            node_text.append(node)
            node_color.append(props['color'])
            node_size.append(props['size'])

        with self.fig.batch_update():
            self.fig.data[0].x = edge_x
            self.fig.data[0].y = edge_y
            self.fig.data[2].x = node_x
            self.fig.data[2].y = node_y
            self.fig.data[2].text = node_text
            self.fig.data[2].marker.color = node_color
            self.fig.data[2].marker.size = node_size
            self.fig.data[2].marker.symbol = 'square'
            self.fig.data[1].x = []
            self.fig.data[1].y = []

        self.update_metrics()

    def handle_node_click(self, trace, points, selector):
        if not points.point_inds: return
        idx = points.point_inds[0]
        if idx >= len(self.node_list): return
        clicked_node = self.node_list[idx]

        if self.remove_toggle.value:
            self.remove_node(clicked_node)
            return

        mode = self.mode_dropdown.value

        if self.source_node is None:
            self.source_node = clicked_node
            self.update_highlights()

            if mode == 'Broadcast':
                self.status_label.value = f"Source: {clicked_node}. Broadcasting..."
                self.calculate_route()
            elif mode == 'Anycast':
                self.status_label.value = f"Source: {clicked_node}. Routing to nearest Spine..."
                self.calculate_route()
            else:
                self.status_label.value = f"Source: {clicked_node}. Select Destination."

        elif mode in ['Unicast', 'Multicast']:
            if clicked_node != self.source_node:
                if clicked_node not in self.target_nodes:
                    self.target_nodes.append(clicked_node)
                    self.update_highlights()

                if mode == 'Unicast':
                    self.status_label.value = f"Route: {self.source_node} -> {clicked_node}"
                    self.calculate_route()
                else:
                    self.status_label.value = f"Multicast: {len(self.target_nodes)} targets."
                    self.calculate_route()

    def remove_node(self, node_name):
        if node_name in self.G:
            self.G.remove_node(node_name)
        if node_name in self.pos:
            del self.pos[node_name]
        self.reset_selection(None)
        self.refresh_plot_data()
        self.status_label.value = f"Node '{node_name}' removed. Metrics updated."

    def calculate_route(self):
        mode = self.mode_dropdown.value
        edges_to_animate = []

        try:
            if mode == 'Unicast' and self.target_nodes:
                path = nx.shortest_path(self.G, source=self.source_node, target=self.target_nodes[0])
                edges_to_animate = list(zip(path, path[1:]))

            elif mode == 'Broadcast':
                edges_to_animate = list(nx.bfs_edges(self.G, source=self.source_node, depth_limit=4))

            elif mode == 'Multicast':
                temp_edges = set()
                sorted_targets = sorted(self.target_nodes, key=lambda t: len(nx.shortest_path(self.G, self.source_node, t)))
                for target in sorted_targets:
                    path = nx.shortest_path(self.G, self.source_node, target)
                    for u, v in zip(path, path[1:]):
                        edge = tuple(sorted((u, v)))
                        if edge not in temp_edges:
                            temp_edges.add(edge)
                            edges_to_animate.append((u, v))

            elif mode == 'Anycast':
                spines = [n for n, attr in self.G.nodes(data=True) if attr.get('type') == 'spine']
                shortest_len = float('inf')
                best_path = []
                for sp in spines:
                    try:
                        p = nx.shortest_path(self.G, self.source_node, sp)
                        if len(p) < shortest_len:
                            shortest_len = len(p)
                            best_path = p
                    except: continue
                if best_path:
                    edges_to_animate = list(zip(best_path, best_path[1:]))

            self.active_edges = edges_to_animate
            self.animation_step = 0
            self.step_btn.disabled = False
            self.animate_step(None)

        except nx.NetworkXNoPath:
            self.status_label.value = "Error: No path found (Network partition detected)."
        except nx.NodeNotFound:
            self.status_label.value = "Error: Selected node no longer exists."

    def animate_step(self, b):
        if self.animation_step > len(self.active_edges): return

        current_slice = self.active_edges[:self.animation_step + 1]
        x_coords, y_coords = [], []

        for u, v in current_slice:
            if u in self.pos and v in self.pos:
                x0, y0 = self.pos[u]
                x1, y1 = self.pos[v]
                x_coords.extend([x0, x1, None])
                y_coords.extend([y0, y1, None])

        with self.fig.batch_update():
            self.fig.data[1].x = x_coords
            self.fig.data[1].y = y_coords

            if self.animation_step >= len(self.active_edges):
                 self.status_label.value = f"Routing Complete ({len(self.active_edges)} hops)."
            else:
                 self.status_label.value = f"Animating... Step {self.animation_step+1}/{len(self.active_edges)}"

        self.animation_step += 1

    def update_highlights(self):
        x, y, color = [], [], []

        if self.source_node and self.source_node in self.pos:
            x.append(self.pos[self.source_node][0])
            y.append(self.pos[self.source_node][1])
            color.append('green')

        for t in self.target_nodes:
            if t in self.pos:
                x.append(self.pos[t][0])
                y.append(self.pos[t][1])
                color.append('red')

        with self.fig.batch_update():
            self.fig.data[3].x = x
            self.fig.data[3].y = y
            self.fig.data[3].marker.line.color = color
            self.fig.data[3].marker.symbol = 'square'

    def reset_selection(self, b):
        self.source_node = None
        self.target_nodes = []
        self.active_edges = []
        self.step_btn.disabled = True
        self.status_label.value = "Simulation Reset. Select Source."

        with self.fig.batch_update():
            self.fig.data[1].x = []
            self.fig.data[1].y = []
            self.fig.data[3].x = []
            self.fig.data[3].y = []

    def on_mode_change(self, change):
        if self.remove_toggle.value: return
        self.reset_selection(None)
        self.status_label.value = f"Mode changed to {change['new']}. Select Source."

    def on_remove_toggle(self, change):
        if change['new']:
            self.status_label.value = "MAINTENANCE MODE: Click a node to remove it."
            self.mode_dropdown.disabled = True
            self.reset_btn.disabled = True
            self.reset_selection(None)
        else:
            self.status_label.value = "Maintenance complete. Resume routing."
            self.mode_dropdown.disabled = False
            self.reset_btn.disabled = False

    def show(self):
        controls = HBox([self.mode_dropdown, self.remove_toggle, self.reset_btn, self.step_btn])
        layout = VBox([controls, self.status_label, self.fig, self.metrics_display])
        display(layout)

In [34]:
sim = NetworkSimulation()
sim.show()